In [1]:
DATA_PATH = 'data'

In [2]:
import pandas as pd
from pathlib import Path

file_path = Path(DATA_PATH) / 'tables_OZ_geo_5700/Карты_мира_озон.xlsm'
source_df = pd.read_excel(file_path)
source_df.head(1)

,SKU,Name,Category,Схема,Brand,Niche,Seller,Balance,Balance FBS,Warehouses count,...,FBS,Base price,Category Position,Categories Last Count,Sales Per Day Average,Turnover,Frozen stocks,Frozen stocks cost,Frozen stocks percent,is_primary
0,936454663,Карта мира географическая политическая интерак...,Канцелярские товары/Бумажная продукция/Карты мира,FBO,NaN,Дом и сад/Декор и интерьер/Картины и панно/Гра...,GooDaY,346,0,12,...,0,4990,1,1,5.2,66.54,112,202832,32.37,False


In [3]:
# Image dataset name
DATASET_NAME = "images_OZ_geo_5700"

In [4]:
# Collect images

import nest_asyncio
nest_asyncio.apply()

from utils import parallel_download_img


# Download image files using the 'Thumb' column for both URL and ID extraction
results = await parallel_download_img(
    df=source_df,
    url_col='Thumb',
    img_id_regex=r'/(\d+)\.jpg$',  # capture numeric ID before '.jpg'
    dataset_name=DATASET_NAME,
    base_download_dir=DATA_PATH,
    max_concurrency=100,
    delay=1.0
)

In [5]:
# Merge the list of results into your DataFrame as a new column

source_df['image_name'] = results

In [8]:
# Save images and names

file_path = (
    Path(DATA_PATH) /
    Path('tables_OZ_geo_5700') /
    Path('OZ_geo_5700_no_descriptions.csv')
)

target_df = (
    source_df[['sku', 'Name', 'image_name', 'url', 'seller']]
    .rename(columns={'Name': 'name'})
)

target_df.to_csv(file_path, index=None)